In [2]:

import requests
import time
import csv
import datetime as dt
import json
import re
import itertools
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
import warnings
import os

from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
from google.colab import auth
from google.colab import drive
from dateutil.parser import parse
from bs4 import BeautifulSoup
from datetime import datetime, date, time

pd.options.display.max_columns = None
pd.options.display.max_rows = None

warnings.filterwarnings("ignore")

drive.mount('/content/drive')


os.makedirs('/content/drive/MyDrive/DenoiseLAB/Task_75_Moex/SmartLab_2022/Firm/', exist_ok=True)
os.makedirs('/content/drive/MyDrive/DenoiseLAB/Task_75_Moex/SmartLab_2023/Firm/', exist_ok=True)
path_save_data_2022_firm = '/content/drive/MyDrive/DenoiseLAB/Task_75_Moex/SmartLab_2022/Firm'
path_save_data_2023_firm = '/content/drive/MyDrive/DenoiseLAB/Task_75_Moex/SmartLab_2023/Firm'
all_data = pd.DataFrame(columns=['date', 'company', 'text'])

# получение_списка_уникальных_авторов_фирм_на_странице
def unique_autors(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    unique_strings = set()
    for tag_i in soup.find_all("i"):
        if tag_i.string:
            unique_strings.add(tag_i.string.strip())
    return unique_strings

def unique_autors_firm(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    unique_strings_autor_firm = set()
    div_tags = soup.find_all("div", class_="cmt_body")
    unique_strings_autor_firm = [div_tag.find("a", class_="a_thread") for div_tag in div_tags]
    for name in range(len(unique_strings_autor_firm)):
        unique_strings_autor_firm[name] = str(unique_strings_autor_firm[name])
        unique_strings_autor_firm[name] = unique_strings_autor_firm[name][:-4]
        unique_strings_autor_firm[name] = unique_strings_autor_firm[name].split('>')[-1]

    return unique_strings_autor_firm

def unique_date_firm(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    unique_strings_date_firm = set()
    div_tags = soup.find_all("div", class_="cmt_body")
    unique_strings_date_firm = [div_tag.find("a", class_="a_time") for div_tag in div_tags]
    for name in range(len(unique_strings_date_firm)):
        unique_strings_date_firm[name] = str(unique_strings_date_firm[name])
        unique_strings_date_firm[name] = unique_strings_date_firm[name][:-33]
        unique_strings_date_firm[name] = unique_strings_date_firm[name].split('datetime="')[-1]

    return unique_strings_date_firm

def unique_autors_message_firm(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    unique_strings_message_firm = set()
    div_tags = soup.find_all("div", class_="text")
    unique_strings_message_firm = [div_tag.get_text() for div_tag in div_tags]
    return unique_strings_message_firm

def get_data_page_firm(start_page, end_page):
    global all_data  # Используем глобальный DataFrame для накопления данных

    for page_number in tqdm(range(start_page, end_page)):
        url_page = f"https://smart-lab.ru/forum/comments/page{page_number}/"

        try:
            response = requests.get(url_page)
            soup = BeautifulSoup(response.content, "html.parser")

            date_col = unique_date_firm(url_page)
            messages_col = unique_autors_message_firm(url_page)
            company_col = unique_autors_firm(url_page)

            # Находим самый короткий список
            shortest_list = min(len(date_col), len(company_col), len(messages_col))
            date_col = date_col[:shortest_list]
            company_col = company_col[:shortest_list]
            messages_col = messages_col[:shortest_list]

            # Создаем DataFrame для текущей страницы
            df = pd.DataFrame({'date': date_col, 'company': company_col,
                             'text': messages_col})

            # Добавляем данные в общий DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)

            # Сохраняем промежуточные результаты каждые 100 страниц
            if page_number % 100 == 0:
                all_data.to_csv(f'{path_save_data_2022_firm}/all_comments.csv', index=False, encoding='utf-8')

        except Exception as e:
            print(f"Ошибка при обработке страницы {page_number}: {str(e)}")
            continue

    return all_data


start_page = 17000
end_page = 18000


all_comments = get_data_page_firm(start_page, end_page)

#в один файл
all_comments.to_csv(f'{path_save_data_2022_firm}/all_comments_final.csv', index=False, encoding='utf-8')

print("Все данные успешно сохранены в один файл.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 1000/1000 [50:03<00:00,  3.00s/it]


Все данные успешно сохранены в один файл.
